![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
<img src="img/columns.png" width=650>

### Terminology

- Database
- Schema
- Queries
- Tables
- Views
- Primary Keys
- Foreign Keys
- Indexes
- Functions & User Defined Functions (UDFs)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=650>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [4]:
cursor.description

(('name', None, None, None, None, None, None),)

In [5]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [6]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [7]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [8]:
# fetchone fetches the next one and then sits there so a subsequent fetchone looks at the next one
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [9]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [10]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [11]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [12]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [13]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Join**

**Where**

**Group by**

**Having**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [15]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

In [18]:
cursor.execute("""
SELECT DISTINCT country
FROM airports
ORDER BY country;
""")
cursor.fetchmany(5)

[('Afghanistan',),
 ('Albania',),
 ('Algeria',),
 ('American Samoa',),
 ('Angola',)]

In [20]:
# list of countrie sorted by # of airports
cursor.execute("""
SELECT country, count(*) as airport_count
FROM airports
GROUP BY country
ORDER BY airport_count DESC;
""")
cursor.fetchmany(5)

[('United States', 1697),
 ('Canada', 435),
 ('Germany', 321),
 ('Australia', 263),
 ('Russia', 249)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [24]:
cursor.execute("""
SELECT *
FROM airports
WHERE id in ('2965', '2990')
""")
print(cursor.fetchall(), sep='\n')

[(2886, '2965', 'Sochi', 'Sochi', 'Russia', 'AER', 'URSS', '43.449928', '39.956589', '89', '4', 'N', 'Europe/Moscow'), (2910, '2990', 'Kazan', 'Kazan', 'Russia', 'KZN', 'UWKD', '55.606186', '49.278728', '411', '4', 'N', 'Europe/Moscow')]


In [31]:
cursor.execute("""
SELECT 
    r.*,
    a_source.latitude source_lat,
    a_source.longitude source_lon,
    a_dest.latitude dest_lat,
    a_dest.longitude dest_lon
FROM routes r
LEFT JOIN airports a_source ON r.source_id = a_source.id
LEFT JOIN airports a_dest ON r.dest_id = a_dest.id
LIMIT 5
;""")
print(cursor.fetchall(), sep='\n')
print([x[0] for x in cursor.description])

[(0, '2B', '410', 'AER', '2965', 'KZN', '2990', None, '0', 'CR2', '43.449928', '39.956589', '55.606186', '49.278728'), (1, '2B', '410', 'ASF', '2966', 'KZN', '2990', None, '0', 'CR2', '46.283333', '48.006278', '55.606186', '49.278728'), (2, '2B', '410', 'ASF', '2966', 'MRV', '2962', None, '0', 'CR2', '46.283333', '48.006278', '44.225072', '43.081889'), (3, '2B', '410', 'CEK', '2968', 'KZN', '2990', None, '0', 'CR2', '55.305836', '61.503333', '55.606186', '49.278728'), (4, '2B', '410', 'CEK', '2968', 'OVB', '4078', None, '0', 'CR2', '55.305836', '61.503333', '55.012622', '82.650656')]
['index', 'airline', 'airline_id', 'source', 'source_id', 'dest', 'dest_id', 'codeshare', 'stops', 'equipment', 'source_lat', 'source_lon', 'dest_lat', 'dest_lon']


### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [33]:
# Which countries have the highest amount of active airlines?
cursor.execute("""
SELECT country, count(*) as airline_count
FROM airlines
WHERE active == "Y"
GROUP BY country
ORDER BY airline_count DESC
LIMIT 5
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)


In [34]:
# Which countries have the highest amount of inactive airlines?
cursor.execute("""
SELECT country, count(*) as airline_count
FROM airlines
WHERE active == "N"
GROUP BY country
ORDER BY airline_count DESC
LIMIT 5
""")
print(*cursor.fetchall(), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)


In [35]:
# What about airports by timezones?
cursor.execute("""
SELECT timezone, count(*) as airport_count
FROM airports
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5
""")
print(*cursor.fetchall(), sep='\n')

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)


In [43]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [36]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [37]:
#Equivalent
max_data = shelter_data.query('Name == "Max"')
max_data.head()
#vs
shelter_data[shelter_data['Name']=="Max"]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
1144,A796351,Max,08/24/2019 12:03:00 PM,08/24/2019 12:03:00 PM,05/31/2010,Transfer,Partner,Cat,Neutered Male,9 years,Domestic Shorthair,Black/White
1174,A618405,Max,08/23/2019 06:38:00 PM,08/23/2019 06:38:00 PM,07/15/2010,Adoption,NaN,Dog,Neutered Male,9 years,American Pit Bull Terrier Mix,White/Blue
1223,A653408,Max,08/23/2019 11:33:00 AM,08/23/2019 11:33:00 AM,05/06/2009,Return to Owner,NaN,Dog,Neutered Male,10 years,American Bulldog Mix,White/Black
1371,A783700,Max,08/20/2019 06:36:00 PM,08/20/2019 06:36:00 PM,11/04/2017,Adoption,NaN,Dog,Neutered Male,1 year,Labrador Retriever Mix,Chocolate/White
2153,A744193,Max,08/08/2019 03:17:00 PM,08/08/2019 03:17:00 PM,09/27/2016,Adoption,Foster,Dog,Neutered Male,2 years,Cairn Terrier Mix,Tan/White
...,...,...,...,...,...,...,...,...,...,...,...,...
106261,A664692,Max,10/27/2013 02:45:00 PM,10/27/2013 02:45:00 PM,10/07/2007,Transfer,Partner,Cat,Neutered Male,6 years,Russian Blue Mix,Blue
106290,A665743,Max,10/26/2013 06:38:00 PM,10/26/2013 06:38:00 PM,12/22/2012,Adoption,NaN,Dog,Neutered Male,10 months,Jack Russell Terrier/Chihuahua Shorthair,White/Brown
106536,A665405,Max,10/21/2013 04:59:00 PM,10/21/2013 04:59:00 PM,06/17/2013,Adoption,NaN,Dog,Neutered Male,4 months,Labrador Retriever Mix,Black/White
107063,A664369,Max,10/12/2013 06:00:00 PM,10/12/2013 06:00:00 PM,05/27/2013,Return to Owner,NaN,Dog,Neutered Male,4 months,Labrador Retriever/German Shepherd,Black


In [38]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,animal_type,Sex upon Outcome,Age upon Outcome,Breed,Color
1,A785262,Calvin,09/13/2019 12:38:00 PM,09/13/2019 12:38:00 PM,04/24/2018,Adoption,NaN,Dog,Neutered Male,1 year,Wire Hair Fox Terrier/Chihuahua Shorthair,Black/White
3,A803789,*Sticks,09/13/2019 12:08:00 PM,09/13/2019 12:08:00 PM,09/06/2017,Adoption,NaN,Dog,Spayed Female,2 years,Cairn Terrier,Tricolor
4,A783086,Pirate,09/13/2019 12:03:00 PM,09/13/2019 12:03:00 PM,10/25/2017,Adoption,Foster,Dog,Neutered Male,1 year,German Shepherd Mix,Brown/White
5,A801743,NaN,09/13/2019 12:03:00 PM,09/13/2019 12:03:00 PM,01/24/2010,Transfer,Partner,Dog,Neutered Male,9 years,Miniature Poodle/Lhasa Apso,Chocolate
6,A804083,NaN,09/13/2019 12:03:00 PM,09/13/2019 12:03:00 PM,09/10/2017,Transfer,Partner,Dog,Intact Female,NaN,Shih Tzu/Miniature Poodle,White


## Goal 6: Transfering from sqlite to pandas

In [40]:
pd_con = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [41]:
# Which countries have the highest amount of active airlines?
df = pd.read_sql_query("""
SELECT country, count(*) as airline_count
FROM airlines
WHERE active == "Y"
GROUP BY country
ORDER BY airline_count DESC
LIMIT 5
""", pd_con)
df

,country,airline_count
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34


In [42]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [2]:
import sqlite3
import pandas as pd

In [15]:
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [20]:
# How many tables are in the database?

# your code here
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
print(tables)
len(tables)

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


11

In [58]:
# What's the primary key of each table?

# your code here
table_info_cols = ['cid', 'name', 'type', 'notnull', 'dflt_value', 'pk']
table_df_cols = ['cid', 'name', 'type', 'notnull', 'dflt_value', 'pk', 'table']
table_df = pd.DataFrame(columns=table_info_cols)
for table in tables:
    cur.execute("PRAGMA table_info("+table[0]+")")
    info = cur.fetchall()
    temp_df = pd.DataFrame(info, columns=table_info_cols)
    temp_df['table'] = table[0]
    table_df = pd.concat([table_df, temp_df])
table_df.loc[table_df.pk==1]

C:\Users\natha\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


,cid,dflt_value,name,notnull,pk,table,type
0,0,None,AlbumId,1,1,Album,INTEGER
0,0,None,ArtistId,1,1,Artist,INTEGER
0,0,None,CustomerId,1,1,Customer,INTEGER
0,0,None,EmployeeId,1,1,Employee,INTEGER
0,0,None,GenreId,1,1,Genre,INTEGER
0,0,None,InvoiceId,1,1,Invoice,INTEGER
0,0,None,InvoiceLineId,1,1,InvoiceLine,INTEGER
0,0,None,MediaTypeId,1,1,MediaType,INTEGER
0,0,None,PlaylistId,1,1,Playlist,INTEGER
0,0,None,PlaylistId,1,1,PlaylistTrack,INTEGER


In [50]:
# What foreign keys join the tables together?

# your code here
# hint: use "PRAGMA foreign_key_list()"
for table in tables:
    print(table[0])
    print(*cur.execute("PRAGMA foreign_key_list("+table[0]+")").fetchall(), sep='\n')


Album
(0, 0, 'Artist', 'ArtistId', 'ArtistId', 'NO ACTION', 'NO ACTION', 'NONE')
Artist

Customer
(0, 0, 'Employee', 'SupportRepId', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')
Employee
(0, 0, 'Employee', 'ReportsTo', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')
Genre

Invoice
(0, 0, 'Customer', 'CustomerId', 'CustomerId', 'NO ACTION', 'NO ACTION', 'NONE')
InvoiceLine
(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE')
(1, 0, 'Invoice', 'InvoiceId', 'InvoiceId', 'NO ACTION', 'NO ACTION', 'NONE')
MediaType

Playlist

PlaylistTrack
(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE')
(1, 0, 'Playlist', 'PlaylistId', 'PlaylistId', 'NO ACTION', 'NO ACTION', 'NONE')
Track
(0, 0, 'MediaType', 'MediaTypeId', 'MediaTypeId', 'NO ACTION', 'NO ACTION', 'NONE')
(1, 0, 'Genre', 'GenreId', 'GenreId', 'NO ACTION', 'NO ACTION', 'NONE')
(2, 0, 'Album', 'AlbumId', 'AlbumId', 'NO ACTION', 'NO ACTION', 'NONE')


In [ ]:
# What are the max and min dates in the Invoice table?
cur.execute("""
-- your code here;
""").fetchall()

In [51]:
df = pd.read_sql_query("SELECT * FROM Invoice;", conn)
df.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [52]:
df = pd.read_sql_query("SELECT MAX(InvoiceDate), MIN(InvoiceDate)\
                        FROM Invoice;", conn)
df.head()

,MAX(InvoiceDate),MIN(InvoiceDate)
0,2013-12-22 00:00:00,2009-01-01 00:00:00


In [60]:
pd.options.display.max_rows = 100
table_df

,cid,dflt_value,name,notnull,pk,table,type
0,0,None,AlbumId,1,1,Album,INTEGER
1,1,None,Title,1,0,Album,NVARCHAR(160)
2,2,None,ArtistId,1,0,Album,INTEGER
0,0,None,ArtistId,1,1,Artist,INTEGER
1,1,None,Name,0,0,Artist,NVARCHAR(120)
0,0,None,CustomerId,1,1,Customer,INTEGER
1,1,None,FirstName,1,0,Customer,NVARCHAR(40)
2,2,None,LastName,1,0,Customer,NVARCHAR(20)
3,3,None,Company,0,0,Customer,NVARCHAR(80)
4,4,None,Address,0,0,Customer,NVARCHAR(70)


In [ ]:
# What tables would you need to answer "what is your most popular track?"

# I propose measuring popularity as the TrackID with the highest count in the InvoiceLine table
# To connect that with an artist, you'd need to join in the AlbumID from the Track table on the TrackID 
# and then join in the ArtistID from the Album table on AlbumID.
# The Artist Name could be joined from the Artist Table on the ArtistID

In [ ]:
# What values from each table?


In [ ]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

results = cur.execute("""
-- your code here;
""").fetchall()

print(*results, sep='\n')

In [72]:
df = pd.read_sql_query("SELECT *, COUNT(*) FROM PlaylistTrack GROUP BY TrackId ORDER BY COUNT(*);", conn)
df.head()

,PlaylistId,TrackId,COUNT(*)
0,1,6,2
1,1,7,2
2,1,8,2
3,1,9,2
4,1,10,2


In [69]:
df = pd.read_sql_query("SELECT *, COUNT(*) FROM InvoiceLIne GROUP BY TrackId ORDER BY SUM(Quantity) DESC LIMIT 20;", conn)
df.head(20)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,COUNT(*)
0,1,1,2,0.99,1,2
1,4,2,8,0.99,1,2
2,581,108,9,0.99,1,2
3,8,3,20,0.99,1,2
4,11,3,32,0.99,1,2
5,14,4,48,0.99,1,2
6,17,4,66,0.99,1,2
7,20,4,84,0.99,1,2
8,1174,215,161,0.99,1,2
9,29,5,162,0.99,1,2


In [74]:
df = pd.read_sql_query("SELECT Track.TrackId, SUM(InvoiceLine.Quantity), Track.Name, Artist.Name \
                        FROM InvoiceLIne\
                        LEFT JOIN Track\
                        USING(TrackId)\
                        JOIN Album\
                        USING(AlbumID)\
                        JOIN Artist\
                        USING(ArtistID)\
                        GROUP BY TrackId\
                        ORDER BY SUM(Quantity) DESC;", conn)
df.head(20)

,TrackId,SUM(InvoiceLine.Quantity),Name,Name
0,2,2,Balls to the Wall,Accept
1,8,2,Inject The Venom,AC/DC
2,9,2,Snowballed,AC/DC
3,20,2,Overdose,AC/DC
4,32,2,Deuces Are Wild,Aerosmith
5,48,2,Not The Doctor,Alanis Morissette
6,66,2,Por Causa De Você,Antônio Carlos Jobim
7,84,2,Welcome Home (Sanitarium),Apocalyptica
8,161,2,Snowblind,Black Sabbath
9,162,2,Cornucopia,Black Sabbath


In [ ]:
# Advanced: get the artist who sang the song!
results = cur.execute("""
-- your code here;
""").fetchall()

print(*results, sep='\n')

In [ ]:
df = pd.read_sql_query("SELECT Artist.Name, Track.TrackId, SUM(InvoiceLine.Quantity), Track.Name\
                        FROM InvoiceLIne\
                        LEFT JOIN Track\
                        USING(TrackId)\
                        GROUP BY TrackId\
                        ORDER BY SUM(Quantity) DESC;", conn)
df

## Reflection

Please fill out today's [Exit Ticket](https://docs.google.com/forms/d/1dZg9MqBmLqJHWupgSBfvhicrVT1vyTejtaz3sXYsgGk/edit)!